In [46]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_validate, cross_val_score, cross_val_predict
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer 
import matplotlib.pyplot as plt 
%matplotlib inline 


In [33]:
train = pd.read_csv("data/train.csv")

In [34]:
train.head()

,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode,Square_ID
0,34.26,-15.91,0.0,887.764222,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3896-14ce-11ea-bce5-f49634744a41
1,34.26,-15.90,0.0,743.403912,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3897-14ce-11ea-bce5-f49634744a41
2,34.26,-15.89,0.0,565.728343,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3898-14ce-11ea-bce5-f49634744a41
3,34.26,-15.88,0.0,443.392774,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c3899-14ce-11ea-bce5-f49634744a41
4,34.26,-15.87,0.0,437.443428,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c389a-14ce-11ea-bce5-f49634744a41


In [35]:
train = train.set_index("Square_ID")

In [36]:
cols_2015 = [col for col in train.columns if "2014" in col or "2015" in col]

In [55]:
def grade_rmse(ytrue, ypred):
    return np.sqrt(mean_squared_error(ytrue, ypred))

In [56]:
my_scorer = make_scorer(grade_rmse, greater_is_better=False)

In [37]:
cols_2015

['target_2015',
 'precip 2014-11-16 - 2014-11-23',
 'precip 2014-11-23 - 2014-11-30',
 'precip 2014-11-30 - 2014-12-07',
 'precip 2014-12-07 - 2014-12-14',
 'precip 2014-12-14 - 2014-12-21',
 'precip 2014-12-21 - 2014-12-28',
 'precip 2014-12-28 - 2015-01-04',
 'precip 2015-01-04 - 2015-01-11',
 'precip 2015-01-11 - 2015-01-18',
 'precip 2015-01-18 - 2015-01-25',
 'precip 2015-01-25 - 2015-02-01',
 'precip 2015-02-01 - 2015-02-08',
 'precip 2015-02-08 - 2015-02-15',
 'precip 2015-02-15 - 2015-02-22',
 'precip 2015-02-22 - 2015-03-01',
 'precip 2015-03-01 - 2015-03-08',
 'precip 2015-03-08 - 2015-03-15']

In [38]:
x_main = train[["X", "Y", "elevation", "LC_Type1_mode", 'precip 2014-11-16 - 2014-11-23',
 'precip 2014-11-23 - 2014-11-30',
 'precip 2014-11-30 - 2014-12-07',
 'precip 2014-12-07 - 2014-12-14',
 'precip 2014-12-14 - 2014-12-21',
 'precip 2014-12-21 - 2014-12-28',
 'precip 2014-12-28 - 2015-01-04',
 'precip 2015-01-04 - 2015-01-11',
 'precip 2015-01-11 - 2015-01-18',
 'precip 2015-01-18 - 2015-01-25',
 'precip 2015-01-25 - 2015-02-01',
 'precip 2015-02-01 - 2015-02-08',
 'precip 2015-02-08 - 2015-02-15',
 'precip 2015-02-15 - 2015-02-22',
 'precip 2015-02-22 - 2015-03-01',
 'precip 2015-03-01 - 2015-03-08',
 'precip 2015-03-08 - 2015-03-15']]

In [39]:
y = train[["target_2015"]]

In [40]:
x_main

,X,Y,elevation,LC_Type1_mode,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2015-01-04 - 2015-01-11,precip 2015-01-11 - 2015-01-18,precip 2015-01-18 - 2015-01-25,precip 2015-01-25 - 2015-02-01,precip 2015-02-01 - 2015-02-08,precip 2015-02-08 - 2015-02-15,precip 2015-02-15 - 2015-02-22,precip 2015-02-22 - 2015-03-01,precip 2015-03-01 - 2015-03-08,precip 2015-03-08 - 2015-03-15
Square_ID,,,,,,,,,,,,,,,,,,,,,
4e3c3896-14ce-11ea-bce5-f49634744a41,34.26,-15.91,887.764222,9,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c3897-14ce-11ea-bce5-f49634744a41,34.26,-15.90,743.403912,9,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c3898-14ce-11ea-bce5-f49634744a41,34.26,-15.89,565.728343,9,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c3899-14ce-11ea-bce5-f49634744a41,34.26,-15.88,443.392774,10,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c389a-14ce-11ea-bce5-f49634744a41,34.26,-15.87,437.443428,10,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c389b-14ce-11ea-bce5-f49634744a41,34.26,-15.86,405.631747,10,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c389c-14ce-11ea-bce5-f49634744a41,34.26,-15.85,389.261362,10,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c389d-14ce-11ea-bce5-f49634744a41,34.26,-15.84,383.123454,10,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000
4e3c389e-14ce-11ea-bce5-f49634744a41,34.26,-15.83,382.920492,10,0.000000,0.000000,0.000000,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.000000


In [31]:
x_main = x_main.set_index("Square_ID")

In [44]:
x_main.head()

,X,Y,elevation,LC_Type1_mode,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2015-01-04 - 2015-01-11,precip 2015-01-11 - 2015-01-18,precip 2015-01-18 - 2015-01-25,precip 2015-01-25 - 2015-02-01,precip 2015-02-01 - 2015-02-08,precip 2015-02-08 - 2015-02-15,precip 2015-02-15 - 2015-02-22,precip 2015-02-22 - 2015-03-01,precip 2015-03-01 - 2015-03-08,precip 2015-03-08 - 2015-03-15
Square_ID,,,,,,,,,,,,,,,,,,,,,
4e3c3896-14ce-11ea-bce5-f49634744a41,34.26,-15.91,887.764222,9,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0
4e3c3897-14ce-11ea-bce5-f49634744a41,34.26,-15.90,743.403912,9,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0
4e3c3898-14ce-11ea-bce5-f49634744a41,34.26,-15.89,565.728343,9,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0
4e3c3899-14ce-11ea-bce5-f49634744a41,34.26,-15.88,443.392774,10,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0
4e3c389a-14ce-11ea-bce5-f49634744a41,34.26,-15.87,437.443428,10,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0


In [43]:
y.head()

,target_2015
Square_ID,
4e3c3896-14ce-11ea-bce5-f49634744a41,0.0
4e3c3897-14ce-11ea-bce5-f49634744a41,0.0
4e3c3898-14ce-11ea-bce5-f49634744a41,0.0
4e3c3899-14ce-11ea-bce5-f49634744a41,0.0
4e3c389a-14ce-11ea-bce5-f49634744a41,0.0


In [45]:
lr = LinearRegression()

In [65]:
cross_validate(lr, x_main, y, cv=5, scoring=my_scorer)

C:\Users\pjd04\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.0129931 , 0.0124979 , 0.01495957, 0.01396132, 0.01595592]),
 'score_time': array([0.00249076, 0.00299644, 0.00299358, 0.00299311, 0.00207186]),
 'test_score': array([-0.13060394, -0.21593088, -0.31629639, -0.14452043, -1.27533719]),
 'train_score': array([-0.22800067, -0.21053019, -0.18386116, -0.2239283 , -0.17824855])}

In [66]:
rfr = RandomForestRegressor(random_state=42)

In [67]:
cross_validate(rfr, x_main, y, cv=5, scoring=my_scorer)

C:\Users\pjd04\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\pjd04\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\pjd04\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\pjd04\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  esti

{'fit_time': array([0.93674397, 0.61336541, 0.9145565 , 0.76196527, 1.05204654]),
 'score_time': array([0.00505757, 0.00794387, 0.00598145, 0.00398469, 0.00698042]),
 'test_score': array([-0.09256783, -0.18754249, -0.16840106, -0.14070642, -0.2944055 ]),
 'train_score': array([-0.04236253, -0.04074161, -0.03926416, -0.04266525, -0.03290352])}